In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import json

with open('transaction-data-adhoc-analysis.json', 'r') as f:
    df = pd.read_json(f)
df.head()

,address,birthdate,mail,name,sex,username,transaction_items,transaction_value,transaction_date
0,USNS Wagner\nFPO AE 00917,1926/03/11,watsonbrenda@gmail.com,Jennifer Campbell,F,veronicawells,"Exotic Extras,Beef Chicharon,(x4)",5196,2022/01/12
1,USCGC Benson\nFPO AP 48794,2016/07/08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"Exotic Extras,Beef Chicharon,(x3);HealthyKid 3...",12056,2022/01/12
2,USCGC Benson\nFPO AP 48794,2016/07/08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"HealthyKid 3+,Gummy Vitamins,(x3);HealthyKid 3...",5500,2022/01/08
3,USCGC Benson\nFPO AP 48794,2016/07/08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"HealthyKid 3+,Yummy Vegetables,(x1)",500,2022/01/10
4,"33911 Hansen Loaf\nWest Sandra, ND 54188",1949/09/27,michael78@yahoo.com,Kristen Cooper,F,qhernandez,"Candy City,Orange Beans,(x3);Candy City,Gummy ...",2697,2022/01/24


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

#Firstly, load the .json file to a DataFrame
with open('transaction-data-adhoc-analysis.json','r') as f:
    boom = pd.read_json(f)

#Function to split multiple transactions by one buyer
def item_split_to_list(transaction_item):
    if ';' in transaction_item:
        item_list = transaction_item.split(';')
        return item_list
    else:
        return [transaction_item]
    
#Function to split the unique items into 'brand | item', and 'qty'
def brand_item(unique_item):
    item_detail_list = unique_item.split(',')
    return (item_detail_list[0]) + ' | ' + item_detail_list[1]
def qty(unique_item):
    item_detail_list = unique_item.split(',')
    return item_detail_list[2]

#Vectorize new function into the 'data' DataFrame (make a new column to preserve the 'transaction_items' col.)
boom['transaction_item_list'] = boom.apply(lambda x: item_split_to_list(x['transaction_items']), axis=1)

#Use .explode() function to expand the list into rows of the same index (will become a numpy array)
transaction_items_exploded = boom['transaction_item_list'].explode()

#Create a new DF to store the numpy array
transaction_items_df = pd.DataFrame()

#Vectorize the 'brand | item' and 'qty' functions
vfunc_brand = np.vectorize(brand_item)
vfunc_qty = np.vectorize(qty)

#Dump the data into the 'transaction_items_df' using the vectorized functions
transaction_items_df['brand | item'] = vfunc_brand(transaction_items_exploded)
transaction_items_df['qty'] = vfunc_qty(transaction_items_exploded)

#Remove unwanted characters in the qty column (only gets the number part)
transaction_items_df['qty'] = transaction_items_df['qty'].str.replace(r'[^0-9]', '',regex=True)

#Convert 'qty' column into integers
transaction_items_df["qty"] = transaction_items_df["qty"].astype(str).astype(int)

#Create a new DataFrame to store unique store items w/o quantity getting in the way
inventory_df = pd.DataFrame(transaction_items_df['brand | item'].unique(), columns=['brand | item'])

#Function that adds the transaction dates per item sold
def transaction_date_per_item(transaction_item_list,transaction_date):
    transaction_dates = []
    for element in transaction_item_list:
        transaction_dates.append(transaction_date)
    return transaction_dates

boom['transaction_date_list'] = boom.apply(lambda x:transaction_date_per_item(x['transaction_item_list'],x['transaction_date']),axis=1)

#Add the exploded 'transaction_date_list' to the transaction_items_df
transaction_dates_exploded = boom['transaction_date_list'].explode(ignore_index=True)
transaction_items_df['transaction_date'] = transaction_dates_exploded

#Split the transaction_date to get the month value easier
transaction_items_df[['year', 'month', 'date']] = transaction_items_df['transaction_date'].str.split('/', expand=True)

#Convert 'year', 'month', and 'date' columns into int
transaction_items_df["year"] = transaction_items_df["year"].astype(str).astype(int)
transaction_items_df["month"] = transaction_items_df["month"].astype(str).astype(int)
transaction_items_df["date"] = transaction_items_df["date"].astype(str).astype(int)

#Get the unit price for each item
dummy_df = pd.DataFrame()
def filter_single_items(trans_item):
    if ';' not in trans_item and '(x1)' in trans_item:
        return trans_item
dummy_df['brand | item']=boom.apply(lambda x:filter_single_items(x['transaction_items']),axis=1)
def filter_unit_price(trans_item, value):
    if ';' not in trans_item and '(x1)' in trans_item:
        return value
dummy_df['unit price']=boom.apply(lambda x:filter_unit_price(x['transaction_items'],x['transaction_value']),axis=1)
dummy_df.dropna(inplace=True)
dummy_df = dummy_df.drop_duplicates(keep='first')
dummy_df['brand | item'] = dummy_df.apply(lambda x:brand_item(x['brand | item']),axis=1)

#Using SQL .join method, we are able to join the two columns based on their unique values
inventory_df = pd.merge(inventory_df,dummy_df, how='inner')
inventory_df

,brand | item,unit price
0,Exotic Extras | Beef Chicharon,1299.0
1,HealthyKid 3+ | Nutrional Milk,1990.0
2,Candy City | Orange Beans,199.0
3,HealthyKid 3+ | Gummy Vitamins,1500.0
4,HealthyKid 3+ | Yummy Vegetables,500.0
5,Candy City | Gummy Worms,150.0
6,Exotic Extras | Kimchi and Seaweed,799.0


In [7]:
#Get the quantity for each item in each month
inventory_list = list(inventory_df['brand | item'])
months = ['Jan','Feb','Mar','Apr','May','Jun']
for i in range(len(months)):
    total_qty_monthly_list = []
    for j in range(len(inventory_list)): # Go over each month
        # Gives a count of each item in the product list
        totay_qty_item_month = transaction_items_df.loc[(transaction_items_df['brand | item'] == inventory_list[j]) & (transaction_items_df['month']== i+1), 'qty'].sum()
        total_qty_monthly_list.append(totay_qty_item_month)
    inventory_df[f'{months[i]} Qty']= total_qty_monthly_list

#Find the total sale per month
def total_sale_per_item(qty,unit_price):
    return qty * unit_price

for i in range(len(months)):
  inventory_df[f'{months[i]} Sale']=inventory_df.apply(lambda x:total_sale_per_item(x[f'{months[i]} Qty'],x['unit price']), axis=1)
inventory_df

,brand | item,unit price,Jan Qty,Feb Qty,Mar Qty,Apr Qty,May Qty,Jun Qty,Jan Sale,Feb Sale,Mar Sale,Apr Sale,May Sale,Jun Sale
0,Exotic Extras | Beef Chicharon,1299.0,9665,10001,9816,9890,10028,9902,12554835.0,12991299.0,12750984.0,12847110.0,13026372.0,12862698.0
1,HealthyKid 3+ | Nutrional Milk,1990.0,9727,9691,9876,9786,9881,9767,19356730.0,19285090.0,19653240.0,19474140.0,19663190.0,19436330.0
2,Candy City | Orange Beans,199.0,9774,10037,9611,9914,9964,10106,1945026.0,1997363.0,1912589.0,1972886.0,1982836.0,2011094.0
3,HealthyKid 3+ | Gummy Vitamins,1500.0,9681,9980,10145,9842,9948,9980,14521500.0,14970000.0,15217500.0,14763000.0,14922000.0,14970000.0
4,HealthyKid 3+ | Yummy Vegetables,500.0,9959,10256,9896,9861,9735,9722,4979500.0,5128000.0,4948000.0,4930500.0,4867500.0,4861000.0
5,Candy City | Gummy Worms,150.0,9559,9996,9986,10043,9801,9934,1433850.0,1499400.0,1497900.0,1506450.0,1470150.0,1490100.0
6,Exotic Extras | Kimchi and Seaweed,799.0,9676,9949,9967,9921,9773,10104,7731124.0,7949251.0,7963633.0,7926879.0,7808627.0,8073096.0
